## Anomaly Detection in MVTec Dataset

Different experiments will be done in this workspace.

Used Models:
* `ConvAE Model`
* `CBAM ConvAE Model`
* `Residual CBAM ConvAE Model`

Batch size: `16`

Epochs: Changes between `500~1000` epochs

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

os.makedirs('utils/', exist_ok=True)
os.chdir('utils')

! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/models.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/losses.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/vision.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/callbacks.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/utils.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/attention_modules.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/backbones.py

os.chdir('/content')
print("Current working directory", os.getcwd())

Current working directory /content


In [4]:
from utils import vision
from utils import utils
from utils import losses
from utils import models
from utils import backbones
from utils import attention_modules
from utils.attention_modules import Conv2DLayerBN, Conv2DLayerRes, ChannelGate, SpatialGate, CBAM

### Experiment Setup Start

In [5]:
# Create experiment folder
EXPERIMENT_NAME = 'CBAM_ConvAE_Model_MVTec'
EXPERIMENT_SAVE_DIR = os.path.join('/content/drive/MyDrive/MASTER/Master_Thesis/Experiments', EXPERIMENT_NAME)
EXPERIMENT_PRELOAD_WEIGHTS_FROM = 'experiment_11'  # ex. 'experiment_2'

In [6]:
experiment = {
    'TYPE': 'train',        # Experiment type: 'train', 'test'
    'ACCELERATOR': 'GPU',   # 'CPU', 'GPU' or 'TPU'

    # Input data
    'DATASET': 'MVTec_metal_nut_Dataset',
    'IMAGE_SIZE': (256, 256),
    'INPUT_SHAPE': (256, 256, 3),
    'PATCH_SIZE': 256,
    'MAX_GRID_NUM': 4,      # Number of maximum grids to be used in partitioning the input image
    'USE_ENTIRE_IMAGES'     : False,  # Use entire/unpartitioned images as input
    'USE_PARTITIONED_IMAGES': True,  # Use partitioned (parameters: PATCH_SIZE, MAX_GRID_NUM) images as input
    'VALID_SIZE': 0.1,      # Validation data size: (Valid Data) / (All Data)
    'DATA_AUG': False,      # Apply data augmentation
    'DATA_AUG_POWER': 2,    # Data augmentation power: How many times data
     # augmentation will be applied to the whole dataset. default 1

    # Model
    'BACKBONE': 'ResCBAM_ConvAE',   # 'ConvAE', 'CBAM_ConvAE' or 'ResCBAM_ConvAE'
    'DECODER_ATTENTION': True,      # Valid if backbone has CBAM. If True, CBAM attention layer in both encoder and decoder layer
    'REDUCTION_RATIO': 16,          # CBAM layer reduction ratio
    'SPARSITY_FACTOR': None,        # The activity regularizer for sparsity in autoencoders - tf.keras.regularizers.l1(0.001)
    'FIRST_TRANIABLE_LAYER_IX': None,  # First trainable layer of pre-trained backbone models - "block4_pool"
    'BATCH_SIZE': 16,               # IF TPU is active set 4, otherwise set anything
    'EPOCHS': 5,
    'OPTIMIZER': tf.keras.optimizers.Adam,
    'LEARNING_RATE': 1e-4,
    'LATENT_DIM': 128,  # set latent dim - shape: (LATENT_DIM, 1) - default 200

    # Loss
    'RECONS_LOSS': losses.ssim_loss, # Reconstruction loss (use tf intrinsic methods: tf.keras.losses.mean_squared_error or losses.ssim_loss)
    'LRELU_SLOPE': 0.2,       # Leaky ReLU activation function slope value
    # Perceptual Loss
    'PERCEPTUAL_LOSS': False, # Use Perceptual loss
    'PERCEPTUAL_LOSS_MODEL': 'ResNet50', # 'custom', 'VGG16', 'VGG19', 'ResNet50' - default 'VGG16'
    'PERCEPTUAL_LAYERS': [35,77,139,150],    # 'conv2_block3_3_conv', 'conv3_block4_3_conv', 'conv4_block6_3_conv', 'conv5_block1_3_conv'
    'PERP_LOSS_LAMBDA': 1,      # Perceptual loss coeff
    'MSE_LOSS_LAMBDA': 0.5,     # MSE coeff

    # Evaluation
    'BIN_MASK_THRSD': 0.5,  # Binary mask threshold
    'MAX_TEST_IMAGES': -1,  # default: -1, Get all images from the testing dataset

    # Save model
    'PRE_LOAD_WEIGHTS': None, # Start to train model with initial weights come from previous training process - Set epoch number (50, 100, etc.)
    'SAVE_WEIGHTS_PER_EPOCH': 5,  # Checkpoints
}

In [7]:
if experiment['TYPE'] == 'train':
    assert(EXPERIMENT_NAME != '...')
    # Create experiment folder
    os.makedirs(EXPERIMENT_SAVE_DIR, exist_ok=True)

    # Model checkpoints will be save in exp_save_dir
    exp_save_dir = utils.create_experimental_output(experiment, EXPERIMENT_SAVE_DIR)

    TRAINING_WEIGHT_DIR = os.path.join(exp_save_dir, 'training_weights')
    # Create folder for checkpoints (training weights)
    os.makedirs(TRAINING_WEIGHT_DIR, exist_ok=True)

    if experiment['PRE_LOAD_WEIGHTS'] != None:
        assert(EXPERIMENT_PRELOAD_WEIGHTS_FROM != None)
        # Start to train model with initial weights come from previous training process
        exp_save_dir = os.path.join(EXPERIMENT_SAVE_DIR, EXPERIMENT_PRELOAD_WEIGHTS_FROM)
        PRE_LOAD_WEIGHT_DIR = os.path.join(exp_save_dir, 'training_weights')
else:  # test mode
    assert(EXPERIMENT_PRELOAD_WEIGHTS_FROM != None)
    # Set experiment save directory and training weight directory manually
    exp_save_dir = exp_save_dir = os.path.join(EXPERIMENT_SAVE_DIR, EXPERIMENT_PRELOAD_WEIGHTS_FROM)
    TRAINING_WEIGHT_DIR = os.path.join(exp_save_dir, 'training_weights')

In [ ]:
print(f"...Experiment {exp_save_dir.split('experiment_')[1]} was initialized...")
print(f"Experiment directory: {EXPERIMENT_SAVE_DIR}")
print(f"Training weights save directory: {TRAINING_WEIGHT_DIR}")

### Experiment Setup End

### Dataset Pre-processing Start

In [ ]:
# MVTec Transistor dataset from Kaggle
# https://www.kaggle.com/datasets/leezhixiong/mvtec-transistor-dataset

! pip install -q kaggle

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d ipythonx/mvtec-ad

utils.unzip_data("/content/mvtec-ad.zip")
! rm /content/mvtec-ad.zip

Set MVTec Dataset Paths for sub datasets

In [11]:
ROOT_PATH = "/content"

# Parse experiment dataset config, get sub dataset folder name
subfolder_name = ((experiment['DATASET'].split("MVTec_")[-1]).split("_Dataset")[0])
TRAIN_DATASET_GOOD_PATH = os.path.join(ROOT_PATH, (subfolder_name + "/train/good"))
TRAIN_DATASET_DEFECT_PATH = None

TEST_DATASET_GOOD_PATH = os.path.join(ROOT_PATH, (subfolder_name + "/test/good"))
TEST_DATASET_DEFECT_PATH = os.path.join(ROOT_PATH, (subfolder_name + "/test"))
GND_DATASET_DEFECT_PATH = os.path.join(ROOT_PATH, (subfolder_name + "/ground_truth"))

In [ ]:
print("TRAIN DATASET GOOD")
# Do not need to separate training dataset into train_good and train_defect (unsupervised learning)
# Do not need to sort train dataset files, they already will be shuffled
train_dataset_files = utils.get_all_img_files_in_directory(TRAIN_DATASET_GOOD_PATH, ext="png", verbose=1)

#print("TEST DATASET GOOD")
#test_dataset_good_files = utils.get_all_img_files_in_directory(TEST_DATASET_GOOD_PATH, ext="png", verbose=1)

print("TEST DATASET DEFECTED")
# Exclude Test Dataset Good Files, get all others
test_dataset_defect_files = utils.get_all_img_files_in_directory(TEST_DATASET_DEFECT_PATH, ext="png", exc="good", verbose=1)
# Sort ground truth dataset according to image file names
test_dataset_defect_files = sorted(test_dataset_defect_files)

print("GROUND TRUTH DATASET DEFECTED")
gnd_dataset_defect_files = utils.get_all_img_files_in_directory(GND_DATASET_DEFECT_PATH, ext="png", verbose=1)
# Sort ground truth dataset according to image file names
gnd_dataset_defect_files = sorted(gnd_dataset_defect_files)

In [13]:
LABELS = ["Defect-free", "Defected"]

Training dataset Validation Split

In [ ]:
# Training/Validation Split
train_valid_separator = len(train_dataset_files) - int(len(train_dataset_files) * experiment['VALID_SIZE'])

print(f"Number of defect free {subfolder_name} images in the training dataset: {len(train_dataset_files[:train_valid_separator])}")
print(f"Number of defect free {subfolder_name} images in the validation dataset: {len(train_dataset_files[train_valid_separator:])}")
print(f"Number of defect free {subfolder_name} images: {len(train_dataset_files[:train_valid_separator])+len(train_dataset_files[train_valid_separator:])}")
print(f"Number of defected {subfolder_name} images in the testing dataset: {len(test_dataset_defect_files)}")
#print(f"Number of defect free {subfolder_name} images in the testing dataset: {len(test_dataset_good)}")

In [ ]:
test_img = utils.load_images(train_dataset_files[1], scl=True)
'''  Uncomment in evaluation notebook
print(f"Test image shape: {test_img.shape}")

plt.imshow(test_img)
plt.axis('off')
'''

Get Modified Image size and number of Width/Height grids

In [ ]:
modified_image_size, grid_width, grid_height = utils.get_new_image_size_according_to_patch_size(test_img.shape[:2],
                                                                                                experiment['PATCH_SIZE'],
                                                                                                max_grid_num=experiment['MAX_GRID_NUM'],
                                                                                                verbose=1)

Data Augmentation Layer

In [17]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

if experiment['DATA_AUG']:
    # Setup data augmentation
    data_aug_layer = Sequential([
      #preprocessing.RandomFlip("horizontal_and_vertical"), # randomly flip images on horizontal/vertical edge
      #preprocessing.RandomRotation(0.2), # randomly rotate images by a specific amount
      #preprocessing.RandomZoom(0.2), # randomly zoom into an image
      tf.keras.layers.RandomBrightness(factor=0.2, value_range=[0.0, 1.0], seed=None),
      # value_range parameter should be [0.0, 1.0] for RandomBrightness
      # if images were scaled before, default value is [0,255]
      tf.keras.layers.RandomContrast(0.2, seed=None),
      #tf.keras.layers.RandomCrop(256, 256, seed=None), Error - Image size changes
      #preprocessing.RandomWidth(0.2), # randomly adjust the width of an image by a specific amount
      #preprocessing.RandomHeight(0.2), # randomly adjust the height of an image by a specific amount
      #preprocessing.Rescaling(1./255) # keep for models like ResNet50V2, remove for EfficientNet
    ], name="data_aug_layer")
else:
    data_aug_layer = None

In [18]:
import random

random.seed(24)

random.shuffle(train_dataset_files)
train_dataset_files = train_dataset_files[:train_valid_separator]

In [ ]:
print(f"Number of {subfolder_name} images in the training dataset: {len(train_dataset_files)}")

Create TF Dataset Pipeline

In [ ]:
# Use partititoned images as input (Parameters: PATCH_SIZE and MAX_GRID_NUM)
if experiment['USE_PARTITIONED_IMAGES']:
    train_dataset = utils.create_dataset_pipeline(train_dataset_files, batch_size=experiment['BATCH_SIZE'], shuffle=True,
                                                  img_size=modified_image_size, scl=True, patch_size=experiment['PATCH_SIZE'],
                                                  entire_img_pathes=experiment['USE_ENTIRE_IMAGES'], aug_layer=data_aug_layer,
                                                  data_aug_power=experiment['DATA_AUG_POWER'], accelerator='GPU')
# Use only entire images dataset as input
else:
    train_dataset = utils.create_dataset_pipeline(train_dataset_files, batch_size=experiment['BATCH_SIZE'], shuffle=True,
                                                      img_size=experiment['IMAGE_SIZE'], scl=True, patch_size=None,
                                                      aug_layer=data_aug_layer, data_aug_power=experiment['DATA_AUG_POWER'],
                                                      accelerator='GPU')

In [ ]:
train_dataset

In [22]:
print("Number of batches to be trained: ", len(train_dataset))

Number of batches to be trained:  50


### Dataset Pre-processing End

In [23]:
'''  Uncomment in evaluation notebook
vision.show_image_samples_from_batch(train_dataset)
'''

'  Uncomment in evaluation notebook\nvision.show_image_samples_from_batch(train_dataset)\n'

### Model Training Start

Create Custom Anomaly Detection Model

In [24]:
if experiment['BACKBONE'] == 'ConvAE':
    # ConvAE Model
    custom_model = backbones.build_ConvAEModelV1(input_shape=experiment['INPUT_SHAPE'],
                                             latent_dim=experiment['LATENT_DIM'],
                                             lrelu_alpha=experiment['LRELU_SLOPE'],
                                             sparsity=experiment['SPARSITY_FACTOR'])
elif experiment['BACKBONE'] == 'CBAM_ConvAE':
    # CBAM ConvAE Model
    custom_model = backbones.build_CBAMConvAEModelV1(input_shape=experiment['INPUT_SHAPE'],
                                                 latent_dim=experiment['LATENT_DIM'],
                                                 reduction_ratio=experiment['REDUCTION_RATIO'],
                                                 attention_for_decoder=experiment['DECODER_ATTENTION'],
                                                 lrelu_alpha=experiment['LRELU_SLOPE'],
                                                 sparsity=experiment['SPARSITY_FACTOR'])
elif experiment['BACKBONE'] == 'ResCBAM_ConvAE':
    # Residual CBAM ConvAE Model
    custom_model = backbones.build_ResCBAMConvAEModelV1(input_shape=experiment['INPUT_SHAPE'],
                                                    latent_dim=experiment['LATENT_DIM'],
                                                    reduction_ratio=experiment['REDUCTION_RATIO'],
                                                    attention_for_decoder=experiment['DECODER_ATTENTION'],
                                                    lrelu_alpha=experiment['LRELU_SLOPE'],
                                                    sparsity=experiment['SPARSITY_FACTOR'])
'''  Uncomment in evaluation notebook
custom_model.summary()
'''

'  Uncomment in evaluation notebook\ncustom_model.summary()\n'

In [25]:
def generate_and_save_images(model, epoch, test_input):
  # All layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  print("test_input.shape: ", test_input.shape)
  print("predictions.shape: ", predictions.shape)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
    plt.subplot(4, 4, i+1)
    plt.imshow(np.array((predictions[i, :, :, :] * 255)).astype(np.uint8))
    plt.axis('off')
    if i >= 15:
      break

  #plt.savefig(experiment['IMGS_DIR'] + '/image_at_epoch_{:d}.png'.format(epoch))
  plt.show()

In [26]:
if experiment['ACCELERATOR'] != 'TPU':
  @tf.function
  def train_step(images):
      with tf.GradientTape() as tape:
          generated_images = custom_model(images, training=True)
          #loss = custom_model.compute_mse_perceptual(images, generated_images)
          loss = custom_model.loss(images, generated_images)

      gradients = tape.gradient(loss, custom_model.trainable_variables)
      custom_model.optimizer.apply_gradients(zip(gradients, custom_model.trainable_variables))

      return loss

In [27]:
from IPython import display
import time

if experiment['ACCELERATOR'] != 'TPU':
  def train(dataset, epochs):
      loss_hist = []  # Keep loss history
      for epoch in range(epochs):
          start = time.time()
          for image_batch in dataset:
              loss = train_step(image_batch)

          loss_hist.append(loss)   # Add loss value to the loss history after each epoch
          print("loss: ", tf.reduce_mean(loss).numpy())

          # Set real epoch value for progressive training process
          real_epoch = epoch
          if experiment['PRE_LOAD_WEIGHTS'] != None:
              real_epoch += experiment['PRE_LOAD_WEIGHTS']

          # Save the model every experiment['SAVE_WEIGHTS_PER_EPOCH'] epochs
          if (real_epoch + 1) % experiment['SAVE_WEIGHTS_PER_EPOCH'] == 0:
            seed = image_batch[:experiment['BATCH_SIZE']]
            display.clear_output(wait=True)
            generate_and_save_images(custom_model,
                                      real_epoch + 1,
                                      seed)

            # Save checkpoints
            utils.save_experiment_checkpoints([custom_model], epoch=(real_epoch+1), save_dir=TRAINING_WEIGHT_DIR)

            print ('Time for epoch {} is {} sec'.format(real_epoch + 1, time.time()-start))

      # Generate after the final epoch
      display.clear_output(wait=True)
      generate_and_save_images(custom_model,
                              real_epoch+1,
                              seed)

      return loss_hist

In [ ]:
if experiment['TYPE'] == 'train':
    custom_model.compile(loss=experiment['RECONS_LOSS'],
                         optimizer=experiment['OPTIMIZER'](learning_rate=experiment['LEARNING_RATE']))

    # Start training the model with previously trained weights
    if experiment['PRE_LOAD_WEIGHTS'] != None:
        # Start to train model with initial weights come from previous training process
        utils.load_model_experiment_weights([custom_model], epoch=experiment['PRE_LOAD_WEIGHTS'], load_dir=PRE_LOAD_WEIGHT_DIR)

    custom_model_hist = train(train_dataset, experiment['EPOCHS'])
else:  # test/inference mode
    # Set load weight epoch number manually
    utils.load_model_experiment_weights([custom_model], epoch=experiment['EPOCHS'], load_dir=TRAINING_WEIGHT_DIR)

In [ ]:
'''  Uncomment in evaluation notebook
if experiment['TYPE'] == 'train':
    utils.remove_training_weights_except_last_epoch(TRAINING_WEIGHT_DIR)  # Remove weights except last epoch's
'''

In [ ]:
if experiment['TYPE'] == 'train':
    plt.plot(custom_model_hist)

### Model Training End

Terminate session programmatically

In [ ]:
from google.colab import runtime
runtime.unassign()